# Aplikovaný Python: Genomová analýza hlavonožců a mlžů

**Kurz:** UAI329 Aplikovaný Python  
**Akademický rok:** 2024/2025  
**Doba řešení:** 1 týden

---

## 📋 Pravidla práce

1. **AI je povolena** (ChatGPT, Claude, Copilot, GitHub AI, atd.)
2. **Kritické myšlení je povinné:** Musíte rozumět každému řádku kódu
3. **Optimalizace je klíčová:** Data jsou velká (30 MB/soubor)
4. **Komentáře a dokumentace:** Váš kód musí být čitelný
5. **Odevzdání:** Tento notebook doplňte a přejmenujte na `JMENO_PRIJMENI_Zadani.ipynb`

**Hodnocení:** Zápočet = část A, Známka = část A + dobrovolná část B

## 👤 Student

**Jméno a příjmení:** ________________________  
**Datum zahájení:** ________________________  
**Datum odevzdání:** ________________________  

**Prohlášení:** Prohlašuji, že jsem tuto práci vypracoval/a samostatně a že rozumím veškerému kódu v tomto notebooku.

**Podpis:** ________________________

## 🧬 Studované druhy

### 1. **Nautilus pompilius** (hlavonožec)
- **Skupina:** Cephalopoda
- **Zvláštnost:** Primitivní, schránka, "živá fosílie"
- **Genom:** ~800 Mb
- **Soubor:** `Nautilus_pompilius_student.fasta`

### 2. **Panopea generosa** (geoduck)
- **Skupina:** Bivalvia (mlž)
- **Zvláštnost:** Největší zavrtávající mlž, extrémní dlouhověkost (až 160 let)
- **Genom:** ~1.2 Gb
- **Soubor:** `Panopea_generosa_student.fasta`

### 3. **Pinctada imbricata** (perlorodka)
- **Skupina:** Bivalvia (mlž)
- **Zvláštnost:** Produkce perletě, biomineralizace
- **Genom:** ~1.0 Gb
- **Soubor:** `Pinctada_imbricata_student.fasta`

### 📁 Adresářová struktura:
<pre>
vase_projekt/
├── data/
│ ├── Nautilus_pompilius_student.fasta # ~30 MB
│ ├── Panopea_generosa_student.fasta # ~30 MB
│ └── Pinctada_imbricata_student.fasta # ~30 MB
├── test_data/
│ └── test_mini.fasta # 1 MB pro testování
└── tento_notebook.ipynb
</pre>                              


## ⚠️ Důležité upozornění

**Soubory jsou velké (30 MB každý)!**  
Před implementací komplexních analýz vždy:

1. **Testujte na malém souboru** (vytvořte si `test.fasta` s 1 scaffoldem)
2. **Měřte výkon** (`%%timeit` v Jupyter)
3. **Kontrolujte spotřebu paměti** (`memory_profiler` nebo `tracemalloc`)

In [ ]:
# Ukázka: Vytvoření testovacího souboru

def create_test_file(input_path, output_path, max_scaffolds=1, max_size=1000000):
    """
    Vytvoří malý testovací soubor z velkého FASTA.
    
    Parametry:
    ----------
    input_path : str
        Cesta k původnímu velkému souboru
    output_path : str
        Cesta pro výstupní testovací soubor
    max_scaffolds : int
        Maximální počet scaffoldů k extrakci
    max_size : int
        Maximální velikost výstupního souboru v bytech
    """
    scaffold_count = 0
    current_size = 0
    
    with open(input_path, 'r') as f_in, open(output_path, 'w') as f_out:
        for line in f_in:
            if line.startswith('>'):
                if scaffold_count >= max_scaffolds:
                    break
                scaffold_count += 1
            
            f_out.write(line)
            current_size += len(line)
            
            if current_size >= max_size:
                break
    
    print(f"✅ Vytvořen testovací soubor: {output_path}")
    print(f"   📊 Scaffoldů: {scaffold_count}")
    print(f"   📏 Velikost: {current_size:,} bytů")

# Příklad použití (odkomentujte pro spuštění):
# create_test_file('data/Nautilus_pompilius_student.fasta', 
#                  'test_mini.fasta', max_scaffolds=1)

# 📊 ČÁST A: ZÁKLADNÍ ÚKOLY (povinné pro zápočet)

---

## 🎯 Úkol A1: Efektivní načítání FASTA souborů

**Cíl:** Implementovat paměťově efektivní načítání velkých genomových dat.

### Požadavky:

**1. Funkce `load_fasta_metadata(filepath)`**
- Vrací slovník `{header: délka_sekvence}`
- **Nenačítá celé sekvenci do paměti!**
- Použijte iterativní čtení souboru

**2. Funkce `read_fasta_chunks(filepath, chunk_size=10000)`**
- Generátor, který vrací dvojice `(header, sequence)` po částech
- Normalizuje sekvence (velká písmena, zachová 'N')
- Parametr `chunk_size` = počet řádků k načtení najednou

### 🔬 Experiment:

- Změřte dobu běhu a spotřebu paměti pro různé `chunk_size` (1000, 10000, 50000)
- Najděte optimální hodnotu pro vaše data

### 📋 Ukázka výstupu pro A1:

```python
# Výstup load_fasta_metadata
metadata = {
    '>JABBOT010000001.1': 1250345,
    '>JABBOT010000002.1': 893456,
    # ...
}

# Výstup read_fasta_chunks (generátor)
for header, seq in read_fasta_chunks('data/test.fasta', chunk_size=10000):
    print(f"{header}: {len(seq)} bp, GC: {calculate_gc(seq):.1f}%")
    break  # jen první scaffold
```

In [ ]:
# %%time
# ZDE IMPLEMENTUJTE ŘEŠENÍ PRO A1

import gzip
import tracemalloc
import time
from typing import Dict, Generator, Tuple

def load_fasta_metadata(filepath: str) -> Dict[str, int]:
    """
    Načte metadata z FASTA souboru bez načítání celých sekvencí do paměti.
    
    Returns:
    --------
    Dict[str, int]
        Slovník {header: délka_sekvence}
    """
    metadata = {}
    # TODO: Implementujte tuto funkci
    # Nápověda: čtěte soubor řádek po řádku
    # Když narazíte na řádek začínající '>', uložte předchozí sekvenci
    # a začněte novou
    
    return metadata

def read_fasta_chunks(filepath: str, chunk_size: int = 10000) -> Generator[Tuple[str, str], None, None]:
    """
    Generátor pro čtení FASTA souboru po částech.
    
    Yields:
    -------
    Tuple[str, str]
        Dvojice (header, sequence) pro každý scaffold
    """
    # TODO: Implementujte tuto funkci
    # Nápověda: Použijte yield pro vracení výsledků po částech
    # Normalizujte sekvence na uppercase, ale zachovejte 'N'
    pass

# Testovací kód (odkomentujte pro testování)
if __name__ == "__main__":
    # Test na malém souboru
    print("🧪 Testování load_fasta_metadata...")
    metadata = load_fasta_metadata("test_mini.fasta")
    print(f"   📦 Načteno {len(metadata)} scaffoldů")
    
    print("\n🧪 Testování read_fasta_chunks...")
    count = 0
    for header, seq in read_fasta_chunks("test_mini.fasta", chunk_size=1000):
        print(f"   🧬 {header[:50]}: {len(seq)} bp")
        count += 1
        if count >= 3:
            break

## 🎯 Úkol A2: Analýza v pohyblivých oknech

**Cíl:** Analyzovat distribuci GC% a N% podél genomu pomocí sliding windows.

### Požadavky:

**1. Funkce `sliding_window_stats(sequence, window_size, step_size)`**
- Vrací seznam slovníků `[{'gc': 45.2, 'n': 1.5, 'pos': 0}, ...]`
- **Použijte efektivní algoritmus** (kumulativní součet nebo deque)
- Sekvence může být dlouhá miliony bp!

**2. Experiment s parametry:**
- Pro **první scaffold z Nautilus** analyzujte s 3 kombinacemi:
  - `window_size=1000, step_size=500` (50% překryv)
  - `window_size=5000, step_size=2500` (50% překryv)
  - `window_size=20000, step_size=10000` (50% překryv)

### 🤔 Otázky k zamyšlení:

- Jak volba parametrů ovlivňuje "hladkost" křivky?
- Která kombinace je nejvhodnější pro detekci ostrých přechodů?
- Kolik oken získáte pro scaffold dlouhý 1M bp s každou kombinací?

### 🧮 Matematické pozadí:

Pro sekvenci délky $L$, velikost okna $W$, krok $S$:

$$
\text{počet oken} = \left\lfloor \frac{L - W}{S} \right\rfloor + 1
$$

$$
\text{GC\%} = \frac{\text{počet G + počet C}}{W} \times 100
$$

$$
\text{N\%} = \frac{\text{počet N}}{W} \times 100
$$

In [ ]:
# %%time
# ZDE IMPLEMENTUJTE ŘEŠENÍ PRO A2

from collections import deque
import numpy as np

def sliding_window_stats(sequence: str, window_size: int, step_size: int) -> list:
    """
    Analyzuje sekvenci pomocí pohyblivých oken.
    
    Parameters:
    -----------
    sequence : str
        DNA sekvence (už normalizovaná na uppercase)
    window_size : int
        Velikost okna v bp
    step_size : int
        Krok posunu okna v bp
    
    Returns:
    --------
    list of dict
        Seznam slovníků s klíči 'gc', 'n', 'pos' pro každé okno
    """
    results = []
    
    # TODO: Implementujte efektivní algoritmus
    # Možnosti:
    # 1. Kumulativní součet (cumsum) pro rychlý výpočet
    # 2. Deque pro posuvné okno
    # 3. Vektorizované řešení s NumPy
    
    return results

def calculate_gc_content(sequence: str) -> float:
    """Vypočítá GC obsah pro danou sekvenci."""
    if not sequence:
        return 0.0
    
    seq_upper = sequence.upper()
    gc_count = seq_upper.count('G') + seq_upper.count('C')
    total_valid = len(seq_upper) - seq_upper.count('N')
    
    if total_valid == 0:
        return 0.0
    
    return (gc_count / total_valid) * 100

def calculate_n_content(sequence: str) -> float:
    """Vypočítá N obsah pro danou sekvenci."""
    if not sequence:
        return 0.0
    
    seq_upper = sequence.upper()
    n_count = seq_upper.count('N')
    return (n_count / len(seq_upper)) * 100

# Testovací sekvence
test_sequence = "ATGCCGTAATCGNATGCNN" * 1000  # Dlouhá testovací sekvence

print("🧪 Test sliding_window_stats...")
# TODO: Otestujte s různými parametry
# results = sliding_window_stats(test_sequence, window_size=100, step_size=50)
# print(f"   📊 Počet oken: {len(results)}")
# if results:
#     print(f"   🔍 První okno: GC={results[0]['gc']:.1f}%, N={results[0]['n']:.1f}%")

## 🎯 Úkol A3: Vizualizace a srovnávací analýza

**Cíl:** Porovnat genomovou organizaci mezi třemi studovanými druhy.

### Požadavky:

**1. Analýza nejdelšího scaffoldu pro každý druh:**
- Použijte `window_size=5000, step_size=2500`
- Vytvořte 3-panelový graf pro každý druh:
  - Panel 1: Průběh GC% podél scaffoldu
  - Panel 2: Průběh N% podél scaffoldu
  - Panel 3: Bodový graf GC% vs N% (všechna okna)

**2. Srovnávací tabulka:**

| Druh | Scaffold | Délka | Průměrné GC% | Medián GC% | % oken s N>10 | Max GC% | Min GC% |
|------|----------|-------|--------------|------------|---------------|---------|---------|
| Nautilus | ... | ... | ... | ... | ... | ... | ... |
| Panopea | ... | ... | ... | ... | ... | ... | ... |
| Pinctada | ... | ... | ... | ... | ... | ... | ... |

**3. Interpretace výsledků:**
- Který druh má nejvyšší variabilitu GC?
- Který druh má nejvíce oblastí s vysokým N?
- Jaký je vztah mezi GC% a N%?

In [ ]:
# %%time
# ZDE IMPLEMENTUJTE ŘEŠENÍ PRO A3

import matplotlib.pyplot as plt
import pandas as pd

def analyze_longest_scaffold(filepath: str, window_size: int = 5000, step_size: int = 2500):
    """
    Analyzuje nejdelší scaffold z FASTA souboru.
    
    Returns:
    --------
    dict
        Slovník s výsledky analýzy a daty pro vizualizaci
    """
    results = {
        'species': '',
        'scaffold_id': '',
        'length': 0,
        'gc_stats': [],
        'n_stats': [],
        'positions': [],
        'windows_data': []  # Seznam všech oken pro scatter plot
    }
    
    # TODO: Implementujte
    # 1. Najděte nejdelší scaffold
    # 2. Analyzujte sliding windows
    # 3. Vypočítejte statistiky
    
    return results

def create_comparison_plot(species_data_list):
    """
    Vytvoří srovnávací grafy pro více druhů.
    
    Parameters:
    -----------
    species_data_list : list of dict
        Seznam výsledků od analyze_longest_scaffold pro každý druh
    """
    n_species = len(species_data_list)
    
    fig, axes = plt.subplots(3, n_species, figsize=(5*n_species, 12))
    
    if n_species == 1:
        axes = axes.reshape(3, 1)
    
    for col_idx, species_data in enumerate(species_data_list):
        # Panel 1: GC% průběh
        ax1 = axes[0, col_idx]
        ax1.plot(species_data['positions'], species_data['gc_stats'], 
                'b-', alpha=0.7, linewidth=0.5)
        ax1.set_title(f"{species_data['species']} - GC%")
        ax1.set_xlabel("Pozice (bp)")
        ax1.set_ylabel("GC%")
        ax1.grid(True, alpha=0.3)
        
        # Panel 2: N% průběh
        ax2 = axes[1, col_idx]
        ax2.plot(species_data['positions'], species_data['n_stats'],
                'r-', alpha=0.7, linewidth=0.5)
        ax2.set_title(f"{species_data['species']} - N%")
        ax2.set_xlabel("Pozice (bp)")
        ax2.set_ylabel("N%")
        ax2.grid(True, alpha=0.3)
        
        # Panel 3: Scatter GC% vs N%
        ax3 = axes[2, col_idx]
        windows = species_data['windows_data']
        if windows:
            gc_vals = [w['gc'] for w in windows]
            n_vals = [w['n'] for w in windows]
            ax3.scatter(gc_vals, n_vals, alpha=0.5, s=1)
            ax3.set_title(f"{species_data['species']} - GC% vs N%")
            ax3.set_xlabel("GC%")
            ax3.set_ylabel("N%")
            ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# TODO: Implementujte hlavní analýza
# 1. Pro každý druh najděte a analyzujte nejdelší scaffold
# 2. Vytvořte grafy
# 3. Vytvořte srovnávací tabulku

# 🚀 ČÁST B: ROZŠÍŘENÉ ÚKOLY (dobrovolné, pro vyšší známky)

---

## 🎯 Úkol B1: K-merová analýza GC-bohatých oblastí (+2 body)

**Cíl:** Identifikovat krátké sekvenční motivy charakteristické pro oblasti s vysokým GC obsahem.

### Požadavky:

**1. Identifikace GC-bohatých oblastí v Nautilus:**
- Najděte všechny okna s `GC% > 70%` a zároveň `N% < 5%`
- Spojte sousedící okna do větších oblastí

**2. Implementace `kmer_spectrum(sequence, k=6, use_minimizers=True)`:**
- Vrátí slovník `{kmer: frekvence}`
- Pokud `use_minimizers=True`, použijte minimizery (k=6, w=3)
- Změřte úsporu paměti oproti naivní implementaci

**3. Porovnání k-merových profilů:**
- **GC-bohaté oblasti** (okna s GC>70%)
- **GC-chudé oblasti** (náhodný výběr oken s GC<40%, stejná celková délka)
- Najděte **top 5 k-merů** s nejvyšším enrichment ratio:

$$
\text{ER}(kmer) = \frac{\text{freq}_{\text{GC-rich}}}{\text{freq}_{\text{GC-poor}} + 0.001}
$$

**4. Biologická interpretace:**
- Jsou nalezené k-mery součástí známých motivů? (CpG, promotory)
- Lze výsledky zobecnit na další druhy?

In [ ]:
# %%time
# ZDE IMPLEMENTUJTE ŘEŠENÍ PRO B1 (pokud se rozhodnete řešit)

def find_gc_rich_regions(window_stats, gc_threshold=70.0, n_threshold=5.0):
    """
    Najde GC-bohaté oblasti s nízkým obsahem N.
    """
    # TODO: Implementujte
    pass

def kmer_spectrum(sequence, k=6, use_minimizers=True, w=3):
    """
    Vypočítá k-merové spektrum sekvence.
    """
    kmer_counts = {}
    
    # TODO: Implementujte
    # Pokud use_minimizers=True, použijte algoritmus minimizerů
    
    return kmer_counts

def calculate_enrichment(rich_kmers, poor_kmers):
    """
    Vypočítá enrichment ratio pro každý k-mer.
    """
    enrichment = {}
    
    # TODO: Implementujte výpočet ER
    
    return enrichment

## 🎯 Úkol B2: Pokročilá optimalizace a paralelizace (+2 body)

**Cíl:** Výrazně zlepšit výkon analýzy pro všechny tři soubory.

### Požadavky:

**1. Vektorizace pomocí NumPy:**
- Přepište jádro `sliding_window_stats` pomocí NumPy
- Použijte `np.lib.stride_tricks.sliding_window_view`
- Změřte zrychlení

**2. Paralelizace:**
- Použijte `multiprocessing` nebo `concurrent.futures`
- Paralelně analyzujte různé scaffoldy
- Změřte zrychlení na 2 a 4 procesech

**3. Streamování s generátory:**
- Upravte pipeline tak, aby data protékala bez ukládání do paměti
- Ukládejte výsledky přímo do CSV souboru

**4. Srovnání metod:**
- Vytvořte tabulku s dobou běhu pro každou optimalizaci
- Diskutujte trade-off mezi složitostí a výkonem

In [ ]:
# %%time
# ZDE IMPLEMENTUJTE ŘEŠENÍ PRO B2 (pokud se rozhodnete řešit)

import numpy as np
from multiprocessing import Pool
import csv
import time

def sliding_window_numpy(sequence, window_size, step_size):
    """
    Vektorizovaná verze sliding_window_stats pomocí NumPy.
    """
    # TODO: Implementujte pomocí NumPy
    pass

def parallel_analysis(filepath, num_processes=4):
    """
    Paralelní analýza všech scaffoldů v souboru.
    """
    # TODO: Implementujte paralelizaci
    pass

# 📈 HODNOCENÍ

---

## ✅ Kritéria pro zápočet (část A):

- [ ] **A1:** Funkční načítání FASTA s měřením výkonu
- [ ] **A2:** Implementace sliding windows s 3 kombinacemi parametrů
- [ ] **A3:** Kompletní sada grafů a tabulka pro všechny 3 druhy
- [ ] **Kód:** Čitelný, komentovaný, bez zbytečných chyb

---

## 🏆 Známkování:

| Známka | Požadavky |
|--------|-----------|
| **A (výborně)** | Část A perfektně + obě části B s kvalitními optimalizacemi a interpretací |
| **B (velmi dobře)** | Část A perfektně + jedna část B nebo obě s menšími nedostatky |
| **C (dobře)** | Část A splněna v plném rozsahu |
| **D (vyhovující)** | Část A splněna s menšími nedostatky |
| **F (nevyhovující)** | Část A nesplněna |

---

## ⭐ Bonusové body:

- **+0.5 bodu:** Elegantní a efektivní kód s pokročilými Python konstrukcemi
- **+0.5 bodu:** Hluboká biologická interpretace výsledků
- **+0.5 bodu:** Kreativní vizualizace nad rámec zadání
- **+0.5 bodu:** Detekce a řešení edge cases (prázdné sekvence, malá okna, atd.)

# 🤖 REFLEXE POUŽITÍ AI

---

## Vyplňte prosím:

### 1. Jak jste používal/a AI nástroje?

- [ ] Generování kostry kódu
- [ ] Debugging chyb  
- [ ] Návrh optimalizací
- [ ] Vysvětlení konceptů
- [ ] Generování komentářů
- [ ] Jiné: ________________

### 2. Které části byly nejužitečnější s AI?

*Zde napište odpověď...*

### 3. Které části jste musel/a řešit zcela manuálně a proč?

*Zde napište odpověď...*

### 4. Co vás AI naučila o Pythonu/bioinformatice?

*Zde napište odpověď...*

### 5. Jaký je váš názor na použití AI ve výuce?

*Zde napište odpověď...*

# 🎓 ZÁVĚREČNÉ SHRNUTÍ

---

## 🧠 Co jste se naučili?

1. **Práce s velkými biologickými daty:** FASTA formát, efektivní načítání
2. **Algoritmy pro genomová analýza:** Sliding windows, k-merová analýza
3. **Optimalizace v Pythonu:** Vektorizace, paralelizace, generátory
4. **Vizualizace vědeckých dat:** Matplotlib, multi-panelové grafy
5. **Srovnávací genomika:** Analýza rozdílů mezi druhy

---

## 🚀 Další kroky (volitelné):

- Přidejte analýzu dalších druhů z vašich dat
- Implementujte detekci CpG ostrovů
- Vytvořte webovou aplikaci pro vizualizaci výsledků
- Publikujte data na GitHub jako ukázkový projekt

---

## 📎 Přílohy

### 🔗 Užitečné odkazy:

- [FASTA format specification](https://en.wikipedia.org/wiki/FASTA_format)
- [Matplotlib tutorial](https://matplotlib.org/stable/tutorials/index.html)
- [NumPy vectorization](https://numpy.org/doc/stable/user/absolute_beginners.html)
- [BioPython](https://biopython.org/) (můžete použít, ale musíte rozumět)

### 📅 Ukázkové řešení bude zveřejněno: 14 dní po deadline

### 📞 Kontakt pro dotazy:

- Email: [vaše emailová adresa]
- Konzultační hodiny: [dny a časy]

---

## 🎉 Hodně štěstí!

*"The important thing is not to stop questioning. Curiosity has its own reason for existence."*  
*— Albert Einstein*

In [1]:
# ZDE MŮŽETE PŘIDAT DALŠÍ BUŇKY PODLE POTŘEBY

# Tato buňka zobrazí informace o použité verzi Pythonu a knihovnách
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl

print(f"🐍 Python: {sys.version}")
print(f"🔢 NumPy: {np.__version__}")
print(f"📊 Pandas: {pd.__version__}")
print(f"📈 Matplotlib: {mpl.__version__}")

🐍 Python: 3.12.3 (main, Nov  6 2025, 13:44:16) [GCC 13.3.0]
🔢 NumPy: 1.26.2
📊 Pandas: 2.3.2
📈 Matplotlib: 3.8.2
